In [2]:
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup

headers = { 'Accept-Encoding': 'gzip, deflate, sdch', 'Accept-Language': 'en-US,en;q=0.8', 'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Cache-Control': 'max-age=0', 'Connection': 'keep-alive',}

def download(st, end):

  data = []

  for i in tqdm(range(st,end)):

    link = 'https://www.goodreads.com/book/show/' + str(i)
    r = requests.get(link, headers=headers, allow_redirects=False)
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      title           = soup.find('h1').text.strip()
    except:
      title = np.nan
    try:
      author_detail  = '|'.join([i.text for i in soup.find('div', class_ = 'ContributorLinksList').find_all('a')])
      author_link    = '|'.join([i.get('href') for i in soup.find('div', class_ = 'ContributorLinksList').find_all('a')])
    except:
      author_detail = np.nan
      author_link   = np.nan
    try:
      rating = soup.find('div', class_ = 'BookPageMetadataSection__ratingStats').find('div', class_ = 'RatingStatistics__rating').text.strip()
    except:
      rating = np.nan
    try:
      rat_rev = soup.find('div', class_ = 'RatingStatistics__meta').get('aria-label')
    except:
      rat_rev = np.nan
    try:
      desc = soup.find('div', class_ = 'BookPageMetadataSection__description').text.strip()
    except:
      desc = np.nan
    try:
      genre = ','.join([i.find('span').text.strip() for i in soup.find('div', class_ = 'BookPageMetadataSection__genres').find_all('span', class_ = 'BookPageMetadataSection__genreButton')])
    except:
      genre = np.nan
    try:
      page = [i.text.strip() for i in soup.find('div', class_ = 'FeaturedDetails').find_all('p') if i.get('data-testid') == 'pagesFormat']
    except:
      page = np.nan
    try:
      year  = [i.text.strip() for i in soup.find('div', class_ = 'FeaturedDetails').find_all('p') if i.get('data-testid') == 'publicationInfo']
    except:
      year = np.nan


    data.append([i, title, rating, rat_rev, page, year ,author_detail, genre, desc, author_link])

  df = pd.DataFrame(data, columns = ['id', 'title', 'rating', 'rat_rev', 'page', 'year' ,'author_detail',
                                    'genre', 'desc', 'author_link'])

  df.to_csv('data/' + str(st) + '_' + str(end) + '.csv', index = False)

In [ ]:
start = 400000

n = 5000
pr = 10

process = []

for i in range(start,start+n*pr,n):
  process.append(multiprocessing.Process(target = download, args = ( i , i + n)))

for p in process:
  p.start()

for p in process:
  p.join()

In [ ]:
import os
csvs = [csv for csv in os.listdir('data') if '.csv' in csv]

df = pd.DataFrame()

for csv in tqdm(csvs):
    df_ = pd.read_csv('data/' + csv )
    df_['category_'] = csv.split('.csv')[0]
    df = pd.concat((df,df_))

df.to_csv('data.csv', index = False)

100%|██████████| 8/8 [00:00<00:00, 16.02it/s]
